In [1]:
import json
import pandas as pd
import numpy as np
import torch
from torch import nn
from pprint import pprint
import jsonlines
import os
import re
from tqdm.notebook import tqdm
import pickle

from typing import *

In [2]:
train_path = 'data/train.jsonl'
dev_path = 'data/dev.jsonl'

# Utils

In [3]:
def save_dictionary(dictionary, path):
    with open(path, 'wb') as f:
        pickle.dump(dictionary, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_dictionary(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# Create word embedding with GloVe

In [4]:
def sentence2vector(sentence: str) -> Optional[torch.Tensor]:
    sentences_word_vector = [word_vectors[w] for w in sentence.split(' ') if w in word_vectors]
    
    if len(sentences_word_vector) == 0:
        return None

    sentences_word_vector = torch.stack(sentences_word_vector)  # tensor shape: (#words X #features)
    return torch.mean(sentences_word_vector, dim=0)

In [5]:
def cosine_similarity(v1: torch.Tensor, v2: torch.Tensor) -> float:
    num = torch.sum(v1 * v2)
    den = torch.linalg.norm(v1) * torch.linalg.norm(v2)
    return (num / den).item()

In [6]:
word_vectors = dict()
n_words = 400_000
with open('embeddings/glove.6B.300d.txt') as f:
    for i, line in tqdm(enumerate(f), total=n_words):

        word, *vector = line.strip().split(' ')
        vector = torch.tensor([float(c) for c in vector])
        
        word_vectors[word] = vector

  0%|          | 0/400000 [00:00<?, ?it/s]

# Dataset class using GloVe

In [7]:
class WiCDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path: str):
        self.data = []
        self.create_dataset(dataset_path)
    
    def create_dataset(self, dataset_path: str) -> None:
        with jsonlines.open(dataset_path, 'r') as f:
            for i, line in enumerate(f.iter()):
                # s1 = sentence2vector(line['sentence1'])
                # s2 = sentence2vector(line['sentence2'])
                sentence = f"{line['sentence1']} {line['sentence2']}"
                sentence_vector = sentence2vector(sentence)
                
                label = torch.tensor(1, dtype=torch.float32) if line['label'] == 'True' else torch.tensor(0, dtype=torch.float32)
                self.data.append((sentence_vector, label))


    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.data[idx]

# Model Class

In [8]:
class MLP(nn.Module):
    def __init__(
        self,
        n_features: int,
        num_layers: int,
        hidden_dim: int,
        activation: Callable[[torch.Tensor], torch.Tensor],
    ) -> None:
        super().__init__()

        self.first_layer = nn.Linear(in_features=n_features, out_features=hidden_dim)

        self.layers = (
            nn.ModuleList()
        )

        for i in range(num_layers):
            self.layers.append(
                nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
            )
        self.activation = activation

        self.last_layer = nn.Linear(in_features=hidden_dim, out_features=1)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, meshgrid: torch.Tensor) -> torch.Tensor:
        """
        Applies transformations to each (x, y) independently

        :param meshgrid: tensor of dimensions [..., 2], where ... means any number of dims
        """
        out = meshgrid

        out = self.first_layer(
            out
        )  # First linear layer, transforms the hidden dimensions from `n_features` (embedding dimension) to `hidden_dim`
        for layer in self.layers:  # Apply `k` (linear, activation) layer
            out = layer(out)
            out = self.activation(out)
        out = self.last_layer(
            out
        )  # Last linear layer to bring the `hiddem_dim` features to a binary space (`True`/`False`)
        
        out = self.sigmoid(out)
        return out.squeeze(-1)


# Training process

In [14]:
def correctly_predicted(predicted, gt):
    predicted_labels = (predicted > 0.5).float()

    return (predicted_labels == gt).sum().item(), gt.shape[0]

def step(model, criterion, xb, yb, opt=None):
    loss = criterion(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)


def fit(epochs, model, criterion, opt, train_dl, valid_dl):
    for epoch in tqdm(range(epochs)):
        model.train()
        for xb, yb in train_dl:
            xb = xb#.to(device)
            yb = yb#.to(device)
            step(model, criterion, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            # losses, nums = zip(*[step(model, criterion, xb, yb) for xb, yb in valid_dl])
            losses = []
            nums = []
            corrects = []
            for xb, yb in valid_dl:
                loss, num = step(model, criterion, xb, yb)
                correct, _ = correctly_predicted(model(xb), yb)
                losses.append(loss)
                nums.append(num)
                corrects.append(correct)
                

        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        val_acc = np.sum(corrects) / np.sum(nums)

        print(f"{epoch} \t {val_loss} \t {val_acc}")


In [ ]:
train_dataset = WiCDataset(train_path)
val_dataset = WiCDataset(dev_path)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.BCELoss()#.to(device)
model = MLP(n_features=300,
            num_layers=3, 
            hidden_dim=30, 
            activation=torch.nn.functional.relu)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
fit(50, model, criterion, optimizer, train_loader, val_loader)

  0%|          | 0/50 [00:00<?, ?it/s]

0 	 0.6933204913139344 	 0.5
1 	 0.6870300517082214 	 0.553
2 	 0.6791563992500305 	 0.579
3 	 0.6789726295471191 	 0.577
4 	 0.6777903146743774 	 0.571
5 	 0.6719452929496765 	 0.585
6 	 0.681974398612976 	 0.559
7 	 0.6650348825454712 	 0.599
8 	 0.6677598247528076 	 0.595
9 	 0.6697707805633545 	 0.588
10 	 0.6627034282684326 	 0.612
11 	 0.675067795753479 	 0.581
12 	 0.6737104334831238 	 0.585
13 	 0.6685144004821777 	 0.615
14 	 0.6694449524879456 	 0.603
15 	 0.6813583598136902 	 0.579
16 	 0.6789848222732544 	 0.58
17 	 0.6840828213691711 	 0.58
18 	 0.6755523300170898 	 0.611
19 	 0.6998516340255737 	 0.592
20 	 0.7067382183074952 	 0.581
21 	 0.7074954133033753 	 0.57
22 	 0.694884271144867 	 0.588
23 	 0.6757417006492614 	 0.601
24 	 0.6978282237052917 	 0.588
25 	 0.6946180810928345 	 0.602
26 	 0.7257387251853943 	 0.603
27 	 0.7149130010604858 	 0.592
28 	 0.707258373260498 	 0.602
29 	 0.7276634707450866 	 0.594
30 	 0.7384644708633423 	 0.573
31 	 0.7264115376472473 	 0